# Восстановление золота из золотосодержащей руды

### Описание

Компания "Цифра" разрабатывает решения для эффективной работы промышленных предприятий. Одной из задач является необходимость научиться предсказывать коэффициент восстановления золота из руды.

### Заказчик:
- "Цифра"

### Исходные данные:

- данные о технологическом процессе и параметрах этапов проведения технологического процесса очистки золота

### Задача:
- построить модель предсказания коэффициента восстановления золота из руды.

### План работы:
- Загрузить и подготовить (предобработать) данные. 
- проанализировать данные.
- построить модель.
- Описать выводы.

### Содержание

## <a href='#section01'>1. Изучение общей информации</a>
* <a href='#section1'>1. Загрузка библиотек и функций, чтение информации</a>
* <a href='#section2'>2. Просмотр данных</a>
* <a href='#section3'>3. Выявление аномалий</a>
* <a href='#section4'>4. Вывод</a>

## <a href='#section02'>2. Исследовательская часть</a>
* <a href='#section21'>1. Предобработка</a>
* <a href='#section22'>2. Анализ изменения концентрации металлов</a>
* <a href='#section23'>3. Анализ распределения размеров гранул сырья</a>
* <a href='#section24'>4. Анализ суммарной концентрации всех веществ на разных стадиях</a>
* <a href='#section25'>5. Вторичная предобработка</a>
* <a href='#section26'>6. Подготовка данных</a>
* <a href='#section27'>7. Обучение модели</a>

## <a href='#section03'>3. Итоговый вывод</a>

### Описание данных:

#### Признаки
- date — дата проведения операции
- все остальные бесконечные и непонятные признаки

#### Целевой признак
- rougher.output.recovery - эффективность обогащения чернового концентрата
- final.output.recovery — эффективноть обогащения финального концентрата

<hr style="border: 1px solid #000;">

## <a id=section01>1. Изучение общей информации</a>

### <a id=section1>1. Загрузка библиотек и функций, чтение информации</a>

Импортирую все необходимые для проекта библиотеки

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib as mpl
import statsmodels.api as sm

from scipy import stats as st
import datetime as dt
import math

from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsRegressor
    
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import SCORERS as sc

Определяю константы, которые будут участвовать в проекте

In [ ]:
BOOTSTRAP_SIZE = 5000   
SHAPIRO_LIMIT = 5000    
state = np.random.RandomState(12345)    

Настраиваю параметры отображения данных

In [ ]:
pd.set_option('display.max_columns', 90)
pd.set_option('display.max_rows', 90)
mpl.rcParams['axes.titlesize'] = 15
mpl.rcParams['axes.titleweight'] = 2

Импортирую библиотеки и вывожу команды для исключения предупреждений.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger().setLevel(logging.CRITICAL) 

Загрузил библиотеки и необходимые настройки. Теперь определяю функции, которые необходимы для решения задач проекта

In [ ]:
def missing_value(df):
    count = []
    missing_value = pd.DataFrame(columns=['NaN_part', 'empty_counts', 'space_counts', '0_counts', '-1_counts', 'unique_counts',\
                                          'min_value', 'max_value', 'dupl_sum', 'dtypes', 'length'], index=df.columns)
    for i in df.columns: 
        missing_value['NaN_part'][i] = df[i].isnull().mean()
        missing_value['empty_counts'][i] = df[df[i] == ''][i].count()
        missing_value['space_counts'][i] = df[df[i] == ' '][i].count()
        missing_value['0_counts'][i] = df[(df[i] == 0)][i].count()
        missing_value['-1_counts'][i] = df[df[i] == -1][i].count()
        missing_value['unique_counts'][i] = len(df[i].unique())
        missing_value['min_value'][i] = df[i].min()
        missing_value['max_value'][i] = df[i].max()
        missing_value['dupl_sum'][i] = df[i].duplicated().sum()
        missing_value['dtypes'][i] = df[i].dtypes
        missing_value['length'][i] = len(df[i])
        
    for i in missing_value.columns:
        count.append(missing_value[missing_value[i] != 0][i].count())

    missing_value.loc['count_not_0'] = count
    return(missing_value)

Функция поиска аномалий в данных

In [ ]:
def introduction_data(list_df, trigger):
    for df in list_df:
        try:
            if trigger == 0:
                print('-----------------------------------------------------------')
                print('Данные о таблице {}:'.format(df.name))
                print('-----------------------------------------------------------')
                print(df.info())
            if trigger == 1:
                print('Данные о таблице: {}'.format(df.name))
                display(df.head())
            if trigger == 2:
                print('Данные о таблице: {}'.format(df.name))
                display(df.describe())
        except:
            print('ERROR! Выберите один из 3 вариантов:')
            print('вывести info() - выбери 0')
            print('вывести первые 5 строк - выбери 1')
            print('вывести describe() - выбери 2')

Функция просмотра данных

In [ ]:
def plot_inspection(df, nrows=10, ncols=4, bins=50, trigger=1):
    fig = plt.figure(figsize=(0.8*nrows, 15*ncols), facecolor=('gray')) 
    #fig.suptitle('Распределение и ящик с усами признаков', fontsize=16)
    plt.subplots_adjust(wspace=0.3, hspace=0.7)
    subplots = 1

    for feature in df.columns:
        if trigger == 1:
            ax = fig.add_subplot(nrows, ncols, subplots)
            ax = df[feature].hist(bins=bins, color='y', density=True, alpha=0.9)
            ax.axvline(x=df[feature].mean(), color='b')
            ax.axvline(x=df[feature].median(), color='r')    
            ax.set_title(df[feature].name, fontsize=10)
            ax.set_facecolor('#eafff5')
            ax.grid()
            subplots +=1
        elif trigger == 0:
            ax = fig.add_subplot(nrows, ncols, subplots)
            sns.boxplot(data=df, x=df[feature], color='b', ax=ax)   
            ax.set_title(df[feature].name.title(), fontsize=14)
            ax.set_facecolor('#eafff5')
            ax.grid(None)
            subplots +=1
        else:
            print('Trigger may be only 0 or 1. Please, change it')

    plt.show()

Функция визуального просмотра одного датасета

In [ ]:
def plot_inspection_2(df, df2, nrows=10, ncols=4, bins=50, trigger=1):
    fig = plt.figure(figsize=(0.8*nrows, 15*ncols), facecolor=('gray')) #надо подобрать грамотно коэффициент
    plt.subplots_adjust(wspace=0.3, hspace=0.7)
    subplots = 1

    for feature in df.columns:
        if trigger == 1:
            ax = fig.add_subplot(nrows, ncols, subplots)
            ax = df[feature].hist(bins=bins, color='y', density=True, alpha=0.9)
            df2[feature].hist(bins=bins, color='g', density=True, alpha=0.9)   
            ax.set_title(df[feature].name, fontsize=10)
            ax.legend(['df1', 'df2'], loc='upper right')
            ax.set_facecolor('#eafff5')
            ax.grid()
            subplots +=1
        elif trigger == 0:
            ax = fig.add_subplot(nrows, ncols, subplots)
            sns.boxplot(data=df, x=df[feature], color='b', ax=ax)   
            ax.set_title(df[feature].name.title(), fontsize=14)
            ax.set_facecolor('#eafff5')
            ax.grid(None)
            subplots +=1
        else:
            print('Trigger may be only 0 or 1. Please, change it')

    plt.show()

Функция визуального просмотра двух датасетов на одном графике

In [ ]:
def hypothesis_2genpop(sample_1, sample_2, alpha=0.05, param_test=True): 
    print(f'Количество наблюдений в выборках на входе: {len(sample_1)}, {len(sample_2)}')
    if param_test == True:
        if len(sample_1) > SHAPIRO_LIMIT:
            sample1 = bootstrap_median(sample_1, BOOTSTRAP_SIZE, SHAPIRO_LIMIT)
        if len(sample_2) > SHAPIRO_LIMIT:
            sample2 = bootstrap_median(sample_2, BOOTSTRAP_SIZE, SHAPIRO_LIMIT)
        if (len(sample_1) <= SHAPIRO_LIMIT) and (len(sample_2) <= SHAPIRO_LIMIT):
            sample1 = sample_1
            sample2 = sample_2

        shapiro_test1 = st.shapiro(sample1.dropna())
        shapiro_test2 = st.shapiro(sample2.dropna())
        print(f'Тест Шапиро-Уилка показал: тест1 {shapiro_test1}, тест2 {shapiro_test2}')
        
        levene_test = st.levene(sample_1.dropna(), sample_2.dropna())
        equal_var = True if (levene_test.pvalue >= alpha) else False
        print(f'equal_var={equal_var}')

        results = st.ttest_ind(sample_1.dropna(), sample_2.dropna(), equal_var=equal_var)
        hypothesis = 'H0' if (results.pvalue >= alpha) else 'H1' if (results.pvalue < alpha) else None
        print(f'Статистический параметрический тест для 2 независимых выборок: {hypothesis}')
      
    elif param_test == False:            
        mann_test = st.mannwhitneyu(sample_1.dropna(), sample_2.dropna())
        hypothesis = 'H0' if (mann_test.pvalue >= alpha) else 'H1' if (mann_test.pvalue < alpha) else None
        print(f'Статистический непараметрический тест для 2 независимых выборок: {hypothesis}')
    else:
        print('Параметр param_test может иметь значения только True or False')

Функция определения статистического различия выборок (pd.Series)

In [ ]:
def hypothesis_2genpop_list(df1, df2, alpha=0.05, param_test=True):
    print(f'Количество наблюдений в выборках на входе: {len(df1)}, {len(df2)}, \
          количество признаков: {len(df1.columns), len(df2.columns)} \n')
    
    if param_test == True:
        test_value = pd.DataFrame(columns=['shapiro_test1', 'shapiro_test2','equal_var', 'ttest'], index=df1.columns)
        for feature in df1.columns:
            if len(df1) > SHAPIRO_LIMIT:
                sample1 = bootstrap_median(df1[feature], BOOTSTRAP_SIZE, SHAPIRO_LIMIT)
            if len(df2) > SHAPIRO_LIMIT:
                sample2 = bootstrap_median(df2[feature], BOOTSTRAP_SIZE, SHAPIRO_LIMIT)
            if (len(df1) <= SHAPIRO_LIMIT) and (len(df2) <= SHAPIRO_LIMIT):
                sample1 = df1[feature]
                sample2 = df2[feature]

            shapiro_test1 = st.shapiro(sample1.dropna())
            shapiro_test2 = st.shapiro(sample2.dropna())
            test_value['shapiro_test1'][feature] = shapiro_test1
            test_value['shapiro_test2'][feature] = shapiro_test2

            levene_test = st.levene(df1[feature].dropna(), df2[feature].dropna())
            equal_var = True if (levene_test.pvalue >= alpha) else False
            test_value['equal_var'][feature] = equal_var

            results = st.ttest_ind(df1[feature].dropna(), df2[feature].dropna(), equal_var=equal_var)
            test_value['ttest'][feature] = 'H0' if (results.pvalue >= alpha) else 'H1' if (results.pvalue < alpha) else None

    elif param_test == False:
        test_value = pd.DataFrame(columns=['mann_test'], index=df1.columns)      
        for feature in df1.columns:
            mann_test = st.mannwhitneyu(df1[feature].dropna(), df2[feature].dropna())
            test_value['mann_test'][feature] = 'H0' if (mann_test.pvalue >= alpha) else 'H1' if (mann_test.pvalue < alpha) else None
    else:
        print('Параметр param_test может иметь значения только True or False')
        
    display(test_value)

Функция определения статистического различия групп выборок (pd.DataFrame)

In [ ]:
def efficiency(C, F, T, df):
    rougher_efficiency = C * (F - T) / (F * (C - T)) * 100
    MAE = mean_absolute_error(df, rougher_efficiency)
    print(f'Средняя абсолютная ошибка MAE = {MAE}')

Функция расчета эффективности

In [ ]:
def bootstrap_median(sample, count, n):
    values = []
    for i in range(count):
        subsample = sample.sample(n=n, replace=True, random_state=state) 
        values.append(subsample.median())
    return(pd.Series(values))

Функция распределения медиан в любой выборке по методу bootstrap

Все функции определены, выгружаю данные

In [ ]:
try:    
    train = pd.read_csv('datasets/gold_recovery_train.csv')
    test = pd.read_csv('datasets/gold_recovery_test.csv')
    full = pd.read_csv('datasets/gold_recovery_full.csv')
except:
    train = pd.read_csv('/datasets/gold_recovery_train.csv')
    test = pd.read_csv('/datasets/gold_recovery_test.csv')
    full = pd.read_csv('/datasets/gold_recovery_full.csv')

### <a id=section2>2. Просмотр данных</a>

In [ ]:
train.name = 'train'
test.name = 'test'
full.name = 'full'
list_df = [train, test, full]

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
full.info()

In [ ]:
introduction_data(list_df, 1)

In [ ]:
introduction_data(list_df, 2)

### <a id=section3>3. Выявление аномалий</a>

Посмотрю наличие аномалий в данных

In [ ]:
value = missing_value(train)
display(value)

In [ ]:
value = missing_value(test)
display(value)

In [ ]:
value = missing_value(full)
display(value)

Посмотрю качественные характеристики нормальности распределения. Для этого переведу в индекс признак 'date'. Он необходим для проведения качественного  и количественного тестов. Создам копию, чтобы можно было обращаться в будущем к исходным данным

In [ ]:
full_copy = full.copy()
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
for df in [full, train, test]:
    df.set_index('date', inplace=True)

Создам датасет full размером с train. Это необходимо будет для проведения статистических тестов

In [ ]:
part_full = full.loc[train.index]
#part_train = train.loc[test.index] 

В датасете test даты отличны от других датасетов, они смещены на 1 сек. Поэтому я не смог создать выборку train размера выборки test. Надо будет проверить даты на нерваность временного ряда и одинаковый шаг.

In [ ]:
plot_inspection(full, nrows=22, ncols=4, bins=50)

In [ ]:
plot_inspection_2(full, train, 22, 4)

Странно, визуально final.output.recovery имеет нормальное распределение, но qq plot не подтверждает этого.

Визуально распределения концентрации металлов на финальном этапе в наборах данных full and train похожи, различаются моды и есть небольшие смещения.

Посмотреть на различия между датасетами train and test я пока не могу - необходимо разобраться с датами, они не совпадают.

Посмотрю количественные характеристики нормальности распределения статистическими параметрическими тестами для независимых выборок

In [ ]:
%%time
hypothesis_2genpop_list(full, train)

- если рассмотреть все тот же признак final.output.recovery, то статистические параметрические тесты тоже не показали нормальность распределения. Хотя визуально ряд признаков выглядят вполне как нормальное распределение.
- тесты Шапиро-Уилка показал значимое различие многих признаков от распределения Гаусса.

Как результат, статистические параметрические тесты нельзя использовать, практически все распределения признаков имеют значимое отличие от Гауссова. Посмотрю на непараметрические тесты для независимых выборок.

In [ ]:
hypothesis_2genpop_list(full, train, param_test=False)

Непараметрический тест Манна-Уитни для независимых выборок в целом показал, что распределения значимо отличаются. То есть они не являются нормальными, и при этом имеют значимых различия между собой. Исключением являются только признаки:

- primary_cleaner.input.depressant
- primary_cleaner.output.tail_ag
- rougher.output.concentrate_pb
- rougher.output.tail_pb
- rougher.state.floatbank10_b_air
- rougher.state.floatbank10_c_air
- rougher.state.floatbank10_c_level
- rougher.state.floatbank10_e_level
- secondary_cleaner.output.tail_pb
- secondary_cleaner.state.floatbank2_b_level

Необходимо убедиться в том, что признаки не коррелируют между собой. Для этого посмотрю на корреляцию данных:

In [ ]:
train_corr_stacked = train.corr().unstack().sort_values(ascending=False).reset_index()
train_corr_stacked.columns=['features_1', 'features_2', 'corr']

collinear_features = train_corr_stacked.query('corr > 0.7')
collinear_features

Я рассматриваю производственный процесс, в котором операции проходят последовательно. Поэтому корреляция должна быть между этапами. 

### <a id=section4>4. Вывод</a>

Аномалии:
    
- наличие NaN
- количество NaN повторяемо по признакам, скорее всего для рада признаков имеют одну причину
- наличие множества выбросов
- все распределения не являются Гауссовым по Q-Q plot и статистическим параметрическим тестам. Но визуально распределения кажутся нормальными. Непараметрические статистические тесты показывают, что ряд признаков имеет одинаковость распределения
- наличие скошенности
- наличие эксцессов, которые визуально подтверждаются не для всех признаков
- date имеет тип данных object
- временные смещения в датасете test
- возможные разные временные шаги и временые разрывы
- мультиколлинеарность, которую возможно вообще не стоит обрабатывать, так как производственный процесс линеен и зависит от предыдущих этапов. 

Вывод:
    
- обработать NaN практически во всех признаках всех столбцов
- изменить тип данных признака date (или в индекс его)
- проверить неразрывность и одношаговость даты в test
- параметрические статистические тесты нельзя использовать, надо пробовать непараметрические
- спросить у Заказчина более подробно информацию

<hr style="border: 1px solid #000;">

## <a id=section02>2. Исследовательская часть</a>

### п. 1.2. Проверю эффективность обогащения на этапе флотации на обучающей выборке. 

In [ ]:
F = train['rougher.input.feed_au']         #доля золота в сырье/концентрате до флотации
C = train['rougher.output.concentrate_au'] #доля золота в концентрате после флотации
T = train['rougher.output.tail_au']        #доля золота в отвальных хвостах после флотации

# efficiency(C, F, T, train['rougher.output.recovery'])

Я скрыл в комментарии последнюю строчку, так как не получилось рассчитать эффективность из-за обилия расчетных NaNов. Они не находятся в датасете и являются результатом вычисления. Причина скорее всего в том, что в оригинальном наборе данных из-за невозможности расчета эффективности обогащения была произведена замена некоторых наблюдений на константные величины. Таким образом был решен вопрос с узким местом склеивания реальных данных с формулой.

Надо оценить эффективность без учета этих данных. Для это произведу удаление этих наблюдений.

In [ ]:
train_drop = train.dropna(how='any', subset=['rougher.output.recovery'])

F = train_drop['rougher.input.feed_au']         #доля золота в сырье до флотации
C = train_drop['rougher.output.concentrate_au'] #доля золота в концентрате после флотации
T = train_drop['rougher.output.tail_au']        #доля золота в отвальных хвостах после флотации

efficiency(C, F, T, train_drop['rougher.output.recovery'])

MAE практически не отличается, поэтому можно утверждать, что эффективность была посчитана по заданной формуле. Кроме того, там где в наборе данных были NaNы эффективность считалась скорее всего подставлением константных переменных исходя из каких-то профессиональных соображений.

Вообще анализ формулы эффективности обогащения приводит к следующим выводам:
- recovery -> 0 => все золото ушло в хвост, что говорит о низкой эффективности обогащения.
- recovery -> 1 => имею идеальную очистку. Примеси уходят в хвост, а золото остается.
- recovery -> ∞ => золота уже не было на этапе очистки, процесс идет вхолостую.

Обобщу результат работы:
    
- предобработка выполнена
- выявлены большие хвосты выбросов практически для всех признаков всех наборов данных. При этом существуют правые хвосты
- есть ложные наблюдения, к примеру концентрация золота после этапа первичной очистки равна 0, а на финальном этапе имеет численное значение.
- есть данные, расчетных характеристики от которых нельзя вычислить, но в датасетах они имеют численное значение. Назову их скрытыми NaN.

### п. 1.3. Выведу признаки, которых нет в тестовой выборке

In [ ]:
train.columns.difference(test.columns)

In [ ]:
# columns_in_test = [i for i in train.columns if i in test.columns]
# print(pd.Index(columns_in_test))

Признаки, которые не вошли в тестовую выборку можно разбить на блоки:
    
- final.output
- primary_cleaner.output
- secondary_cleaner.output
- rougher.calculation
- rougher.output

Вошедшие в выборку test признаки:
    
- primary_cleaner.input
- весь state
- rougher.input

Можно сделать вывод, что в выборке test данные о сырье, реагентах и текущем состоянии параметров этапов: объема воздуха, уровня жидкости, размер гранул сырья и скорости подачи сырья. В test нет выходных данных output и расчетных характеристик.

### <a id=section21>1. Предобработка</a>

Проверю временной интервал на разрывность и одношаговость. Так как в оригинале я дату перевел в индекс, то посмотрю в копии датасетов

In [ ]:
full_copy.name = 'full'
train_copy.name = 'train'
test_copy.name = 'test'

In [ ]:
for df in [full_copy, train_copy, test_copy]:
    df['date'] = pd.to_datetime(df['date'])
    time_min = df['date'].diff().min()
    time_max = df['date'].diff().max()
    print(f'В датасете {df.name} шаг составляет max={time_max} - min={time_min} = {time_max - time_min}')

- в выборке full and train шаг разнороден: 59:59 мин и 1 час - смещение на 1 сек
- в выборке test шаг однороден - 1 час
- в датасете train и test есть временные разрывы. Длительность разрыва достигает 4 и 8 месяцев

Скорее всего разрывы обусловлены тем, что в train and test добавлялись наблюдения из full не блоком, а случайным образом.

Я заменю все пропуски в train and test исходя из тех соображений, что full является их прародителем. Если после разделения train and test были изменены, то мой подход не правилен. 

In [ ]:
train.update(full, overwrite=False)
test.update(full, overwrite=False)

Теперь заполню оставшиеся пропуски методами bfill() and ffill()

In [ ]:
train = train.ffill().bfill()
test = test.ffill().bfill()

Проверю результат замены пропусков

In [ ]:
value = missing_value(train)
value[0:len(value):len(value)-1]

In [ ]:
value = missing_value(test)
value[0:len(value):len(value)-1]


Промежуточный вывод
    
- есть разрыв и смещение в дате
- в train and test from full данные передавались случайным образом
- часть пропусков в train and test заполнил из full
- заполнил остальные пропуски методами ffill() and bfill()

### <a id=section22>2. Анализ изменения концентрации металлов</a>

### п. 2.1. Посмотрю, как меняется концентрация металлов на различных этапах очистки

In [ ]:
concentrate = {'ag':['rougher.output.concentrate_ag', 'primary_cleaner.output.concentrate_ag', 'final.output.concentrate_ag' ],
               'au':['rougher.output.concentrate_au', 'primary_cleaner.output.concentrate_au', 'final.output.concentrate_au' ],
               'pb':['rougher.output.concentrate_pb', 'primary_cleaner.output.concentrate_pb', 'final.output.concentrate_pb' ],
               'sol':['rougher.output.concentrate_sol', 'primary_cleaner.output.concentrate_sol', 'final.output.concentrate_sol']}

fig = plt.figure(figsize=(16, 5), facecolor=('gray')) 
fig.suptitle(f'Распределение значений концентрации металлов на разных этапах ', fontsize=16)
plt.axis('off')
subplots = 1

for key in concentrate:
    ax = fig.add_subplot(1, 4, subplots)
    sns.boxplot(data=full[concentrate[key]], color='b', ax=ax)   
    ax.set_title(f'Концентрация {key}', fontsize=14)
    ax.set_facecolor('#eafff5')
    ax.grid(None)
    ax.set_xticklabels([])
    subplots +=1
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 5), facecolor=('gray')) 
fig.suptitle(f'Распределение значений концентрации металлов на разных этапах ', fontsize=16)
plt.subplots_adjust(wspace=0.3)
subplots = 1

for key in concentrate:
    ax = fig.add_subplot(1, 4, subplots)
    for i in range(0, 3):
        ax = full[concentrate[key][i]].plot(kind='hist', bins=50, grid=True) 
        ax = plt.axvline(x=full[concentrate[key][i]].median(), linewidth=1, color='r', label='mean')
    plt.title(f'Концентрация {key}', fontsize=14)
    subplots += 1
plt.show()

Медиана концентрации золота с каждым этапом увеличивается, примерно с 20 до 45. Это говорит о том, что технологический процесс работает хорошо. Очистка золота положительна. При этом график распределения золота сужается, что так же хорошо - увеличивается устойчивость вероятного прогноза.

Медиана концентрации серебра понижается примерно с 12 до 5. Видимо серебро участвует в химической реакции и является расходным веществом. После второй очистки распределение серебра сужается. 

Медиана свинца после первой очистки повысилась, видимо на этих этапах он активно использовался. После второй очистки медиана практически не изменилась, а распределение относительно нее стало таким же как было на этапе флотации. Можно говорить о некотором скачке медианы распределения концентрации свинца после первой очистки.

Медиана концентрации неизвестного вещества sol понижалась примерно с 30 до 8. Могу предположить, что оно являлось расходным материалом для обогащения золота.

К финальному этапу распределения всех металлов сужаются, повышается устойчивость прогноза, что я расцениваю позитивно. При этом каждое распределение имеет большие хвосты около 0. Предположу, что причиной могут быть:
- ошибочные прогнозы концентрации металлов
- программные ошибки расчетов в технологическом процессе
- ошибочные данные о наличии золота в руде и т. д.

Вывод - хвосты надо обрабатывать.

### <a id=section23>3. Анализ распределения размеров гранул сырья</a>


### п. 2.2 Рассмотрю распределения размеров гранул сырья на обучающей и тестовой выборках.

In [ ]:
feed_train = pd.DataFrame(train[[x for x in train.columns if 'feed_size' in x]])
feed_test = pd.DataFrame(test[[x for x in test.columns if 'feed_size' in x]])

Признаки 2х выборок имеют различные размеры. Состав набора данных также различен. Удалю лишние признаки и возьму из выборки train измерения, соответствующие измерениям выборки test по индексу

In [ ]:
plot_inspection(feed_train, 20, 4, trigger=0)

In [ ]:
plot_inspection(feed_test, 20, 4, trigger=0)

In [ ]:
plot_inspection_2(feed_train, feed_test, 20, 4)

Визуальный анализ распределения размера гранул сырья показывает:
- размахи идентичны на этапе флотации. На этапе первичной обработки разнятся, но на уровне выбросов
- есть выбросы с левой и правой стороны

Посчитаю статистические различия для пар размеров гранул сырья как для независимых выборок. Буду рассматривать выборки одинаковой длины. То есть от выборки train я буду использовать только четверть данных, которые сохранил в датасет part_feed_train

In [ ]:
hypothesis_2genpop(train['rougher.input.feed_size'].dropna(), test['rougher.input.feed_size'].dropna(), param_test=True)

Гипотеза о равенстве двух средних признаков отклонена, статистический параметрический анализ показывает, что средние значимо не равны.

In [ ]:
hypothesis_2genpop(train['rougher.input.feed_size'].dropna(), test['rougher.input.feed_size'].dropna(), param_test=False)

Гипотеза о равенстве двух средних признаков отклонена, статистический непараметрический анализ показывает, что средние также значимо не равны.

Промежуточный вывод:
    
- Сравнение распределения гранул сырья привело к позитивным выводам. Распределения в целом похожи друг на друга. Есть различия в выбросах с разных сторон.
- необходимо обработать выбросы

### <a id=section24>4. Анализ суммарной концентрации всех веществ на разных стадиях</a>

### п.2.3. Исследую суммарную концентрацию всех веществ на разных стадиях

In [ ]:
concentrate_input = pd.DataFrame(full[[x for x in full.columns if 'rougher.input.feed' in x]], index=full.index)
concentrate_rougher = pd.DataFrame(full[[x for x in full.columns if 'rougher.output.concentrate' in x]])
concentrate_primary_cleaner = pd.DataFrame(full[[x for x in full.columns if 'primary_cleaner.output.concentrate' in x]])
concentrate_final = pd.DataFrame(full[[x for x in full.columns if 'final.output.concentrate' in x]])

In [ ]:
concentrate_input = concentrate_input.drop(['rougher.input.feed_rate', 'rougher.input.feed_size'], axis=1)

In [ ]:
concentrate_sum = pd.DataFrame(index=concentrate_input.index)
concentrate_sum['input_sum'] = concentrate_input.sum(axis=1)
concentrate_sum['rougher_sum'] = concentrate_rougher.sum(axis=1)
concentrate_sum['primary_cleaner_sum'] = concentrate_primary_cleaner.sum(axis=1)
concentrate_sum['final_sum'] = concentrate_final.sum(axis=1)

In [ ]:
fig = plt.figure(figsize=(10, 5), facecolor=('gray'))
fig.suptitle(f'Суммарная концентрация всех веществ на разных стадиях')
ax = plt.figure(figsize=(16, 5), facecolor=('gray'))
for i in concentrate_sum:
    ax = concentrate_sum[i].plot(kind='hist', bins=200, legend=True, grid=True)
    ax = plt.axvline(concentrate_sum[i].median(), color='green', linewidth=0.5, label='median')
plt.show()

- медианы суммарной концентрации rougher_sum and final_sum сливаются на графике. Они практически одинаковы

Необходима вторая предобработка:
- удалить экстремальные выбросы
- удалить ложные наблюдения

### <a id=section25>5. Вторичная предобработка</a>

Удалю те значения концентрации веществ на разных стадиях, сумма которых меньше 0.1

In [ ]:
length_train = len(train)
length_test = len(test)

In [ ]:
train.shape

In [ ]:
concentrate_sum_indexes = concentrate_sum[np.sum(concentrate_sum, axis=1) > 0.0].index
train_preprocessed = train[train.index.isin(concentrate_sum_indexes)]

In [ ]:
train_preprocessed.shape

In [ ]:
print(f'было потеряно {(length_train - len(train_preprocessed)) / length_train * 100:.2f}% данных')

In [ ]:
value = missing_value(train_preprocessed)
value[0:len(value):len(value)-1]

Остались еще нулевые значения. Возможно стоит их обработать, но пока не понимаю как

### <a id=section26>6. Подготовка модели</a>

### 3.1. Напишу функции расчета метрики качества

In [ ]:
def smape(target, predict):
    summary = 200 / len(target) * np.sum(abs(target - predict) / (abs(target) + abs(predict))) 
    return(summary)

In [ ]:
def smape_total(rougher_target, rougher_predict, final_target, final_predict):    
    return(0.25 * smape(rougher_target, rougher_predict) + 0.75 * smape(final_target, final_predict))

### Подготовлю модель

Возьму в обучающий датасет те данные, которые в test. Насколько я понял, это необходимо из-за специфики бизнеса

In [ ]:
train = train_preprocessed[test.columns]

Выделю целевые признаки и добавлю их в train и test.

In [ ]:
full_target = full[['final.output.recovery', 'rougher.output.recovery']]
train = pd.concat([train, full_target], join='inner', axis=1)
test = pd.concat([test, full_target], join='inner', axis=1)

удалю пропуски в целевых признаках

In [ ]:
train = train.dropna()
test = test.dropna()

In [ ]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
print(f'было потеряно в train {(length_train - len(train)) / length_train * 100:.2f}% данных')
print(f'было потеряно в test {(length_test - len(test)) / length_test * 100:.2f}% данных')

In [ ]:
value = missing_value(train)
value[0:len(value):27]

In [ ]:
value = missing_value(test)
value[0:len(value):27]

In [ ]:
target_train_rougher = train['rougher.output.recovery']
target_train_final = train['final.output.recovery']
target_test_rougher = test['rougher.output.recovery']
target_test_final = test['final.output.recovery']
features_train = train.drop(['rougher.output.recovery', 'final.output.recovery'], axis=1)
features_test = test.drop(['rougher.output.recovery', 'final.output.recovery'], axis=1)

отмасштабирую данные

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train)

features_train_scaled = pd.DataFrame(scaler.transform(features_train), columns=features_train.columns)
features_test_scaled = pd.DataFrame(scaler.transform(features_test), columns=features_test.columns)
print(features_train_scaled.shape)
print(features_test_scaled.shape)

### <a id=section27>7. Обучение модели</a>

Логично предположить, что обучать rougher надо на данных, полученных до флотации. 

In [ ]:
features_train_rougher = features_train_scaled[[i for i in features_train_scaled.columns if 'rougher' in i]]
features_train_final = features_train_scaled
features_test_rougher = features_test_scaled[[i for i in features_test_scaled.columns if 'rougher' in i]]
features_test_final = features_test_scaled
print(features_train_rougher.columns)

Создам собственную метрику на основе формулы расчета эффективности обогащения золота

In [ ]:
smape_scorer = make_scorer(smape, greater_is_better = False)

Оценю результаты простой модели на обучающей выборке

In [ ]:
%%time
model_dummy_rougher = DummyRegressor(strategy='median')
score_dummy_rougher = abs(cross_val_score(model_dummy_rougher, features_train_rougher, target_train_rougher, cv=1000, \
                                          scoring=smape_scorer).mean())
predict_dummy_rougher = pd.Series(cross_val_predict(model_dummy_rougher, features_train_rougher, target_train_rougher, cv=1000))
print(f'Простая dummy-модель на этапе флотации: {model_dummy_rougher}, score={score_dummy_rougher:.2f}')

model_dummy_final = DummyRegressor(strategy='median')
score_dummy_final = abs(cross_val_score(model_dummy_final, features_train_final, target_train_final, cv=1000, \
                                        scoring=smape_scorer).mean())
predict_dummy_final = pd.Series(cross_val_predict(model_dummy_final, features_train_final, target_train_final, cv=1000))
print(f'Простая dummy-модель на финальном этапе: {model_dummy_final}, score={score_dummy_final:.2f}')

smape_total_dummy = smape_total(target_train_rougher, predict_dummy_rougher, target_train_final, predict_dummy_final)
print(f'Общая эффективность на простой модели smape_total={smape_total_dummy:.2f}')

Обучу модель решающих деревьев 

In [ ]:
%%time
best_model_dtree_rougher = None
best_score_dtree_rougher = 100000
best_predict_dtree_rougher = None

for depth in range(1, 20, 1):
    model_dtree_rougher = DecisionTreeRegressor(random_state=state, max_depth=depth)
    predict_dtree_rougher = pd.Series(cross_val_predict(model_dtree_rougher, features_train_rougher, target_train_rougher, cv=20))
    score_dtree_rougher = abs(cross_val_score(model_dtree_rougher, features_train_rougher, target_train_rougher, cv=20, scoring=smape_scorer).mean())
    
    if score_dtree_rougher < best_score_dtree_rougher:
        best_score_dtree_rougher = score_dtree_rougher
        best_model_dtree_rougher = model_dtree_rougher
        best_predict_dtree_rougher = predict_dtree_rougher

print(f'Лучшая модельDecisionTree на этапе флотации: {best_model_dtree_rougher}, score={best_score_dtree_rougher:.2f}')

best_model_dtree_final = None
best_score_dtree_final = 100000
best_predict_dtree_final = None

for depth in range(1, 20, 1):
    model_dtree_final = DecisionTreeRegressor(random_state=state, max_depth=depth)
    predict_dtree_final = pd.Series(cross_val_predict(model_dtree_final, features_train_final, target_train_final, cv=20))
    score_dtree_final = abs(cross_val_score(model_dtree_final, features_train_final, target_train_final, cv=20, scoring=smape_scorer).mean())
    
    if score_dtree_final < best_score_dtree_final:
        best_score_dtree_final = score_dtree_final
        best_model_dtree_final = model_dtree_final
        best_predict_dtree_final = predict_dtree_final
        
print(f'Лучшая модель DecisionTree на финальном этапе: {best_model_dtree_final}, score={best_score_dtree_final:.2f}')

smape_total_dtree = smape_total(target_train_rougher, best_predict_dtree_rougher, target_train_final, best_predict_dtree_final)
print(f'Общая эффективность на модели DecisionTreeRegressor - smape_total={smape_total_dtree:.2f}')

Проверю модель решающих деревьев на тестовой выборке

In [ ]:
predict_test_dtree_rougher = pd.Series(cross_val_predict(best_model_dtree_rougher, features_test_rougher, target_test_rougher, cv=50))
predict_test_dtree_final = pd.Series(cross_val_predict(best_model_dtree_final, features_test_final, target_test_final, cv=50))

smape_total_dtree_test = smape_total(target_test_rougher, predict_test_dtree_rougher, target_test_final, predict_test_dtree_final)
print(f'на тестовой выборке общая эффективность: {smape_total_dtree_test:.2f}')

Модель решающих деревьев на обучающей и тестововй выборке оказалась лучше простой dummy-модели. Попробую обучить модель случайного леса

In [ ]:
%%time
best_model_forest_rougher = None
best_score_forest_rougher = 10000
best_predict_forest_rougher = None
for est in range(1, 50, 5):
    for depth in range(1, 10, 1):
        model_forest_rougher = RandomForestRegressor(random_state=state, n_estimators=est, max_depth=depth)
        predict_forest_rougher = pd.Series(cross_val_predict(model_forest_rougher, features_train_rougher, target_train_rougher, cv=20))
        score_forest_rougher = abs(cross_val_score(model_forest_rougher, features_train_rougher, target_train_rougher, cv=20, \
                                                  scoring=smape_scorer).mean())
        if score_forest_rougher < best_score_forest_rougher:
            best_score_forest_rougher = score_forest_rougher
            best_model_forest_rougher = model_forest_rougher
            best_predict_forest_rougher = predict_forest_rougher
print(f'Лучшая модель RandomForest на этапе флотации: {best_model_forest_rougher}, score={best_score_forest_rougher:.2f}')

best_model_forest_final = None
best_score_forest_final = 10000
best_predict_forest_final = None
for est in range(1, 50, 5):
    for depth in range(1, 10, 1):
        model_forest_final = RandomForestRegressor(random_state=state, n_estimators=est, max_depth=depth)
        predict_forest_final = pd.Series(cross_val_predict(model_forest_final, features_train_final, target_train_final, cv=20))
        score_forest_final = abs(cross_val_score(model_forest_final, features_train_final, target_train_final, cv=20, \
                                                scoring=smape_scorer).mean())
        if score_forest_final < best_score_forest_final:
            best_score_forest_final = score_forest_final
            best_model_forest_final = model_forest_final
            best_predict_forest_final = predict_forest_final
print(f'Лучшая модель RandomForest на финальном этапе: {best_model_forest_final}, score={best_score_forest_final:.2f}')

smape_total_forest = smape_total(target_train_rougher, best_predict_forest_rougher, target_train_final, best_predict_forest_final)
print(f'Общая эффективность на модели RandomForest - smape_total={smape_total_forest:.2f}')

In [ ]:
predict_test_forest_rougher = pd.Series(cross_val_predict(best_model_forest_rougher, features_test_rougher, target_test_rougher, cv=50))
predict_test_forest_final = pd.Series(cross_val_predict(best_model_forest_final, features_test_final, target_test_final, cv=50))

smape_total_forest_test = smape_total(target_test_rougher, predict_test_forest_rougher, target_test_final, predict_test_forest_final)
print(f'на тестовой выборке общая эффективность: {smape_total_forest_test:.2f}')

Модель случайного леса на обучающей и тестовой выборках оказалась лучше простой dummy-модели. Попробую обучить модель к-ближайших соседей

In [ ]:
%%time
best_model_knn_rougher = None
best_score_knn_rougher = 10000
best_predict_knn_rougher = None
for knn in range(1, 30, 1):
    model_knn_rougher = KNeighborsRegressor(n_neighbors=knn)
    predict_knn_rougher = pd.Series(cross_val_predict(model_knn_rougher, features_train_rougher, target_train_rougher, cv=20))
    score_knn_rougher = abs(cross_val_score(model_knn_rougher, features_train_rougher, target_train_rougher, cv=20, scoring=smape_scorer).mean())
    if score_knn_rougher < best_score_knn_rougher:
        best_score_knn_rougher = score_knn_rougher
        best_model_knn_rougher = model_knn_rougher
        best_predict_knn_rougher = predict_knn_rougher
print(f'Лучшая модель к-случайных соседей {best_model_knn_rougher}, score={best_score_knn_rougher:.2f}')

best_model_knn_final = None
best_score_knn_final = 1000
best_predict_knn_final = None
for knn in range(1, 30, 1):
    model_knn_final = KNeighborsRegressor(n_neighbors=knn)
    predict_knn_final = pd.Series(cross_val_predict(model_knn_final, features_train_final, target_train_final, cv=20))
    score_knn_final = abs(cross_val_score(model_knn_final, features_train_final, target_train_final, cv=20, scoring=smape_scorer).mean())
    if score_knn_final < best_score_knn_final:
        best_score_knn_final = score_knn_final
        best_model_knn_final = model_knn_final
        best_predict_knn_final = predict_knn_final
print(f'Лучшая модель к-случайных соседей {best_model_knn_final}, score={best_score_knn_final:.2f}')

smape_total_knn = smape_total(target_train_rougher, best_predict_knn_rougher, target_train_final, best_predict_knn_final)
print(f'Общая эффективность на модели KNeighborsRegressor - smape_total={smape_total_knn:.2f}')

In [ ]:
predict_test_knn_rougher = pd.Series(cross_val_predict(best_model_knn_rougher, features_test_rougher, target_test_rougher, cv=50))
predict_test_knn_final = pd.Series(cross_val_predict(best_model_knn_final, features_test_final, target_test_final, cv=50))

smape_total_knn_test = smape_total(target_test_rougher, predict_test_knn_rougher, target_test_final, predict_test_knn_final)
print(f'на тестовой выборке общая эффективность: {smape_total_knn_test:.2f}')

Модель к-ближайших соседей на обучающей и тестовой выборке оказалась лучше простой dummy-модели. Попробую обучить линейную модель.

In [ ]:
%%time
model_linear_rougher = LinearRegression()
predict_linear_rougher = pd.Series(cross_val_predict(model_linear_rougher, features_train_rougher, target_train_rougher, cv=1000))
score_linear_rougher = abs(cross_val_score(model_linear_rougher, features_train_rougher, target_train_rougher, cv=1000, \
                                           scoring=smape_scorer).mean())
print(f'Линейная модель на этапе флотации: {model_linear_rougher}, score={score_linear_rougher:.2f}')

model_linear_final = LinearRegression()
predict_linear_final = pd.Series(cross_val_predict(model_linear_final, features_train_final, target_train_final, cv=1000))
score_linear_final = abs(cross_val_score(model_linear_final, features_train_final, target_train_final, cv=1000, \
                                         scoring=smape_scorer).mean())
print(f'Линейная модель на финальном этапе:  {model_linear_final}, score={score_linear_final:.2f}')

smape_total_linear = smape_total(target_train_rougher, predict_linear_rougher, target_train_final, predict_linear_final)
print(f'Общая эффективность на модели RandomForestRegressor - smape_total={smape_total_linear:.2f}')

In [ ]:
predict_test_linear_rougher = pd.Series(cross_val_predict(model_linear_rougher, features_test_rougher, target_test_rougher, cv=1000))
predict_test_linear_final = pd.Series(cross_val_predict(model_linear_final, features_test_final, target_test_final, cv=1000))

smape_total_linear_test = smape_total(target_test_rougher, predict_test_linear_rougher, target_test_final, predict_test_linear_final)
print(f'на тестовой выборке общая эффективность: {smape_total_linear_test:.2f}')

Линейная модель на обучающей и тестовой выборке оказалась лучше простой dummy-модели. Обобщу результаты

In [ ]:
SmapeTotal = pd.Series()
SmapeTotal['dummy'] = smape_total_dummy
SmapeTotal['dtree_train'] = smape_total_dtree
SmapeTotal['forest_train'] = smape_total_forest
SmapeTotal['knn_train'] = smape_total_knn
SmapeTotal['linear_train'] = smape_total_linear
SmapeTotal['dtree_test'] = smape_total_dtree_test
SmapeTotal['forest_test'] = smape_total_forest_test
SmapeTotal['knn_test'] = smape_total_knn_test
SmapeTotal['linear_test'] = smape_total_linear_test
display(SmapeTotal)

Из таблицы видно, что в данном наборе моделей победила простая линейная модель.

## <a id=section03>3. Итоговый вывод</a>

В ходе выполнения работы были использованы модели регрессии: 
- решающих деревьев
- случайного леса
- линейной регрессии
- к-ближайших соседей

Выявлено было следующее:
- Данные оказались c большим количеством пропусков в признаках. 
- распределение признаков значимо отличается от распределения Гаусса
- распределения признаков в датасетах значимо отличаются друг от друга
- данные имеют временные разрывы и не являются временным рядом

Оценка моделей проводилась по уникальной метрике расчета эффективности обогащения руды

В результате исследования я выявил, что на модели линейной регрессии можно добиться наилучшего значения метрики smape_total=8.57. 